In [ ]:
from src.generator import data_generator, clip_generator, label_generator
from src.util import characters, id_from_char, char_from_id
from src.models import create_model

from pymongo import MongoClient
from tensorflow import keras

import numpy as np
import pickle

In [ ]:
for character in characters:
    print(character)

# The Model

Source code for model creation is in src/models.py

In [ ]:
# # to make a new model
# model = create_model()

In [ ]:
# # to load the saved model
# model = keras.models.load_model('models/cnn_model')
# model.summary()

# Testing

In [ ]:
database_name = 'slippi'
collection_name = 'Blynde_clips'

# Connect to the hosted MongoDB instance
client = MongoClient('localhost', 27017)
db = client[database_name]
collection = db[collection_name]
collection.estimated_document_count()

In [ ]:
batch_size = 100
skip = None    # no skip because first 100k clips are test data
step = 1       # to ensure that consecutive clips aren't from the same player and game
repeat = False  # keep this as True, allows generator to loop through database infinitley
limit = 100000 # only look at test data (first 100k clips)

limit = None

num_batches = None

In [ ]:
data_test = data_generator(clip_collection=collection,
                           batch_size = batch_size,
                           skip=skip,
                           step=step,
                           repeat=repeat,
                           limit=limit)

X_test = clip_generator(clip_collection=collection,
                        batch_size = batch_size,
                        skip=skip,
                        step=step,
                        repeat=repeat,
                        limit=limit)

Y_test = label_generator(clip_collection=collection,
                         batch_size = batch_size,
                         skip=skip,
                         step=step,
                         repeat=repeat,
                         limit=limit, 
                         onehot=True)

y_test = label_generator(clip_collection=collection,
                         batch_size = batch_size,
                         skip=skip,
                         step=step,
                         repeat=repeat,
                         limit=limit,
                         onehot=False)

In [ ]:
score = model.evaluate(data_test, steps=num_batches, verbose=1)

print('\nTest score:', round(score[0], 3))
print(f'Test accuracy: {round(score[1]*100)}%')
print(f'Test test top 8 categorical accuracy: {round(score[2]*100)}%') 

# Confusion Matrix

In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
def get_conf_matrix(labels_as_id, predictions_as_id):
    conf_matrix = np.zeros((27,27))
    for i_real, i_pred in zip(labels_as_id, predictions_as_id):
        conf_matrix[i_real, i_pred] += 1
    return conf_matrix

In [ ]:
recalls = {k:0 for k in range(1,27)}
precisions = {k:0 for k in range(1,27)}

pred = np.argmax(model.predict(X_test, steps=num_batches, verbose=1), axis = 1)
labels = np.concatenate([label for label in y_test])

conf_matrix = get_conf_matrix(labels, pred)
conf_df = pd.DataFrame(conf_matrix[1:,1:], index=characters, columns=characters)

In [ ]:
# sn.heatmap(conf_df, annot=False, mask=conf_df <= 0, cmap='viridis')
fig, ax = plt.subplots(figsize=(13,10))
sn.heatmap(np.log(1 + conf_df), annot=False, ax=ax, square=False)
fig.savefig('confusion_matrix.png')

# Training

In [ ]:
database_name = 'slippi'
collection_name = 'melee_clips_30s'

# Connect to the hosted MongoDB instance
client = MongoClient('localhost', 27017)
db = client[database_name]
collection = db[collection_name]

In [ ]:
# skip first 100k clips and save them as test data
data = data_generator(collection, batch_size=100, skip=100000, step=19, repeat=True)

In [ ]:
model.fit(data, epochs=10, steps_per_epoch=500, verbose=1)

score = model.evaluate(data, steps=50, verbose=0)
print('\nTest score:', round(score[0], 3))
print(f'Test accuracy: {round(score[1]*100)}%')
print(f'Test test top 8 accuracy: {round(score[2]*100)}%')

In [ ]:
# model.save('models/cnn_model')